In [ ]:
# Import dependencies

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy import inspect
from datetime import date
import config


## Extract CSVs into DataFrames

In [ ]:
# CSV file #1
# Delete
vaccinations_file = "country_vaccinations.csv"
vaccinations_df = pd.read_csv(vaccinations_file)
vaccinations_df

In [ ]:
# CSV file #2

covid_data_file = "worldwide_covid_data.csv"
covid_data_df = pd.read_csv(covid_data_file)
covid_data_df


## Transform vaccinations DataFrame

In [ ]:
# Get data on the latest date

latest_date = vaccinations_df['date'].max()
latest_date

In [ ]:
# Get data at one timepoint (latest_date) from vaccinations_df

filtered_vaccinations_df = vaccinations_df.loc[vaccinations_df['date'] == latest_date]
filtered_vaccinations_df

In [ ]:
# Generate a new dataframe selecting 2 columns

new_vaccinations_df = filtered_vaccinations_df[["country", "total_vaccinations"]].copy()
new_vaccinations_df


In [ ]:
# Delete NaN values

vaccinations = new_vaccinations_df.dropna(how="any")
vaccinations


## Transform covid_data DataFrame

In [ ]:
# Generate a new covid dataframe

covid_df = covid_data_df[["country", "total_cases", "total_deaths", "population"]].copy()
covid_df

In [ ]:
# Delete NaN values

clean_covid_df = covid_df.dropna(how="any")
clean_covid_df


## Create database connection


In [ ]:

protocol = 'postgresql'
username = config.pg_username
password = config.pg_password
host = 'localhost'
port = 5432
database_name = 'covid-19_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)


In [ ]:
# Confirm tables

insp.get_table_names()


## Load DataFrames into database

In [ ]:

vaccinations.to_sql(name='vaccinations', con=engine, if_exists='append', index=False)


In [ ]:

clean_covid_df.to_sql(name='covid_data', con=engine, if_exists='append', index=False)


## Join the tables

In [ ]:
sql_join = r"""SELECT vaccinations.country, vaccinations.total_vaccinations, 
    covid_data.country, covid_data.total_cases, covid_data.total_deaths, covid_data.population
FROM vaccinations
INNER JOIN covid_data
ON vaccinations.country = covid_data.country"""

pd.read_sql_query(sql_join, con=engine).head()